In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Define paths
train_dir = 'CT_images/Train'
test_dir = 'CT_images/Test'

# Define transformations for training and testing data
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create datasets and loaders
full_train_dataset = datasets.ImageFolder(train_dir, data_transforms['train'])
test_dataset = datasets.ImageFolder(test_dir, data_transforms['test'])

# Split training dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final layer to match binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Set device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate accuracy and collect true labels and predictions
def calculate_accuracy_and_preds(loader, model):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5
            correct += torch.sum(preds == labels.data)
            total += labels.size(0)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    accuracy = correct.double() / total
    return accuracy, np.vstack(all_preds), np.vstack(all_labels)

# Training function with validation accuracy
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            preds = torch.sigmoid(outputs) > 0.5
            correct += torch.sum(preds == labels.data)
            total += labels.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct.double() / total

        # Calculate validation accuracy
        val_acc, _, _ = calculate_accuracy_and_preds(val_loader, model)

        print(f'Epoch {epoch}/{num_epochs - 1} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}')

    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=10)

# Calculate train, validation, and test accuracies
train_acc, _, _ = calculate_accuracy_and_preds(train_loader, model)
val_acc, _, _ = calculate_accuracy_and_preds(val_loader, model)
test_acc, test_preds, test_labels = calculate_accuracy_and_preds(test_loader, model)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Flatten predictions and true labels for confusion matrix calculation
test_preds = test_preds.flatten()
test_labels = test_labels.flatten()

# Calculate confusion matrix and extract TP, TN, FP, FN
cm = confusion_matrix(test_labels, test_preds)
tn, fp, fn, tp = cm.ravel()

# Display confusion matrix and its components
print(f"Confusion Matrix:\n{cm}")
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")

# Generate and display classification report
report = classification_report(test_labels, test_preds, target_names=['Normal', 'Stone'], zero_division=1)
print("Classification Report:")
print(report)


# Save the trained model
torch.save(model.state_dict(), 'resnet18_ct.pth')



C:\Users\Om\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Om\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/9 | Train Loss: 0.1140 | Train Acc: 0.9587 | Val Acc: 0.9683
Epoch 1/9 | Train Loss: 0.0104 | Train Acc: 0.9979 | Val Acc: 0.9983
Epoch 2/9 | Train Loss: 0.0354 | Train Acc: 0.9875 | Val Acc: 0.9800
Epoch 3/9 | Train Loss: 0.0043 | Train Acc: 0.9988 | Val Acc: 1.0000
Epoch 4/9 | Train Loss: 0.0007 | Train Acc: 1.0000 | Val Acc: 1.0000
Epoch 5/9 | Train Loss: 0.0003 | Train Acc: 1.0000 | Val Acc: 1.0000
Epoch 6/9 | Train Loss: 0.0002 | Train Acc: 1.0000 | Val Acc: 1.0000
Epoch 7/9 | Train Loss: 0.0001 | Train Acc: 1.0000 | Val Acc: 1.0000
Epoch 8/9 | Train Loss: 0.0001 | Train Acc: 1.0000 | Val Acc: 1.0000
Epoch 9/9 | Train Loss: 0.0000 | Train Acc: 1.0000 | Val Acc: 1.0000
Train Accuracy: 1.0000
Validation Accuracy: 1.0000
Test Accuracy: 0.9900
Confusion Matrix:
[[600   0]
 [  9 291]]
True Positives (TP): 291
True Negatives (TN): 600
False Positives (FP): 0
False Negatives (FN): 9
Classification Report:
              precision    recall  f1-score   support

      Normal       0

In [3]:
import os
import torch
from PIL import Image
from torchvision import transforms, models
import pandas as pd

# Path to the directory containing the CT images
image_dir = 'CT_images/Test/Stone'

# Define transformation (same as used for testing)
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the trained ResNet18 model
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)  # Binary classification
model.load_state_dict(torch.load('resnet18_ct.pth'))  # Load trained weights
model.eval()  # Set model to evaluation mode

# Set device to GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Function to make prediction for each image
def predict_for_image(image_path, model):
    # Load the image
    image = Image.open(image_path)
    
    # Apply the same transformations as during training
    image_tensor = image_transforms(image).unsqueeze(0)  # Add batch dimension
    image_tensor = image_tensor.to(device)
    
    # Make prediction
    with torch.no_grad():
        output = model(image_tensor)
        prediction = torch.sigmoid(output).item()  # Get the prediction probability
    
    # Return the prediction score (raw probability)
    return prediction

# Function to evaluate all images from "Stone- (1001)" to "Stone- (1300)"
def evaluate_all_images():
    results = []

    # Loop over images from 1001 to 1300
    for i in range(1001, 1301):
        image_name = f'Stone- ({i}).jpg'
        image_path = os.path.join(image_dir, image_name)

        # Check if image exists
        if os.path.exists(image_path):
            # Get prediction score from the model
            prediction_score = predict_for_image(image_path, model)
            
            # Check if the prediction is "Kidney Stone Not Present" (prediction < 0.5)
            if prediction_score < 0.5:
                # Append image name and prediction score to results only if the prediction is "Not Present"
                results.append({
                    "Image Name": image_name,
                    "Prediction Score": round(prediction_score, 4),
                    "Result": "Kidney Stone Not Present"
                })

    return results

# Run evaluation and collect results
predictions = evaluate_all_images()

# Convert results to a pandas DataFrame for tabular display
df_results = pd.DataFrame(predictions)

# Print the results in tabular format
if not df_results.empty:
    print(df_results.to_string(index=False))
else:
    print("No images with 'Kidney Stone Not Present' detected.")

# Optional: Save results to a CSV file
df_results.to_csv('kidney_stone_not_present_results.csv', index=False)

print("Evaluation complete. Results saved to 'kidney_stone_not_present_results.csv'")


C:\Users\Om\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Om\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Om\AppData\Local\Temp\ipykernel_14244\1380716609.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `Tr

       Image Name  Prediction Score                   Result
Stone- (1091).jpg            0.3332 Kidney Stone Not Present
Stone- (1092).jpg            0.3835 Kidney Stone Not Present
Stone- (1093).jpg            0.3129 Kidney Stone Not Present
Stone- (1094).jpg            0.2563 Kidney Stone Not Present
Stone- (1095).jpg            0.1435 Kidney Stone Not Present
Stone- (1096).jpg            0.1169 Kidney Stone Not Present
Stone- (1097).jpg            0.1737 Kidney Stone Not Present
Stone- (1098).jpg            0.3166 Kidney Stone Not Present
Stone- (1099).jpg            0.3872 Kidney Stone Not Present
Evaluation complete. Results saved to 'kidney_stone_not_present_results.csv'
